In [1]:
import pandas as pd 
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
url_2010 = "http://www.pro-football-reference.com/years/2010/draft.htm"

# get the html
html = urlopen(url_2010)

# create the BeautifulSoup object
soup = BeautifulSoup(html, "lxml")

In [3]:
# Extract the necessary values for the column headers from the table
# and store them as a list
column_headers = [th.getText() for th in 
                  soup.findAll('tr', limit=2)[1].findAll('th')]

# Add the two additional column headers for the player links
column_headers.extend(["Player_NFL_Link", "Player_NCAA_Link"])

In [4]:
# The data is found within the table rows of the element with id=draft
# We want the elements from the 3rd row and on
table_rows = soup.select("#drafts tr")[2:]

In [5]:
table_rows[0]

<tr><th class="right" data-stat="draft_round" scope="row">1</th><td class="right" csk="0.5" data-stat="draft_pick">1</td><td class="left" csk="ram01" data-stat="team"><a href="/teams/ram/2010_draft.htm" title="St. Louis Rams">STL</a></td><td class="left" csk="Bradford,Sam" data-append-csv="BradSa00" data-stat="player"><a href="/players/B/BradSa00.htm">Sam Bradford </a></td><td class="left" csk="QB--------0001" data-stat="pos">QB</td><td class="right" data-stat="age">22</td><td class="right" data-stat="year_max">2018</td><td class="right iz" data-stat="all_pros_first_team">0</td><td class="right iz" data-stat="pro_bowls">0</td><td class="right" data-stat="years_as_primary_starter">5</td><td class="right" data-stat="career_av">44</td><td class="right" data-stat="draft_av">24</td><td class="right" data-stat="g">83</td><td class="right" data-stat="pass_cmp">1855</td><td class="right" data-stat="pass_att">2967</td><td class="right" data-stat="pass_yds">19449</td><td class="right" data-stat=

In [6]:
def extract_player_data(table_rows):
    """
    Extract and return the the desired information from the td elements within
    the table rows.
    """
    # create the empty list to store the player data
    player_data = []
    
    for row in table_rows:  # for each row do the following

        # Get the text for each table data (td) element in the row
        # Some player names end with ' HOF', if they do, get the text excluding
        # those last 4 characters,
        # otherwise get all the text data from the table data
        player_list = [td.get_text() for td in row.find_all("th")]
        player_list.extend([td.get_text() for td in row.find_all("td")])

        # there are some empty table rows, which are the repeated 
        # column headers in the table
        # we skip over those rows and and continue the for loop
        if not player_list:
            continue

        # Extracting the player links
        # Instead of a list we create a dictionary, this way we can easily
        # match the player name with their pfr url
        # For all "a" elements in the row, get the text
        # NOTE: Same " HOF" text issue as the player_list above
        links_dict = {(link.get_text()[:-4]   # exclude the last 4 characters
                       if link.get_text().endswith(" HOF")  # if they are " HOF"
                       # else get all text, set thet as the dictionary key 
                       # and set the url as the value
                       else link.get_text()) : link["href"] 
                       for link in row.find_all("a", href=True)}

        # The data we want from the dictionary can be extracted using the
        # player's name, which returns us their pfr url, and "College Stats"
        # which returns us their college stats page
    
        # add the link associated to the player's pro-football-reference page, 
        # or en empty string if there is no link
        player_list.append(links_dict.get(player_list[3], ""))

        # add the link for the player's college stats or an empty string
        # if ther is no link
        player_list.append(links_dict.get("College Stats", ""))

        # Now append the data to list of data
        player_data.append(player_list)
        
    return player_data

In [7]:
# extract the data we want
data = extract_player_data(table_rows)

# and then store it in a DataFrame
df_2010 = pd.DataFrame(data, columns=column_headers)
df_2010.head()

,Rnd,Pick,Tm,Player,Pos,Age,To,AP1,PB,St,...,Rec,Yds,TD,Solo,Int,Sk,College/Univ,,Player_NFL_Link,Player_NCAA_Link
0,1,1,STL,Sam Bradford,QB,22,2018,0,0,5,...,1,5,0,,,,Oklahoma,College Stats,/players/B/BradSa00.htm,http://www.sports-reference.com/cfb/players/sa...
1,1,2,DET,Ndamukong Suh,DT,23,2020,3,5,11,...,0,0,0,368,1,63.5,Nebraska,College Stats,/players/S/SuhxNd99.htm,http://www.sports-reference.com/cfb/players/nd...
2,1,3,TAM,Gerald McCoy,DT,22,2019,1,6,10,...,0,0,0,235,,59.5,Oklahoma,College Stats,/players/M/McCoGe99.htm,http://www.sports-reference.com/cfb/players/ge...
3,1,4,WAS,Trent Williams,T,22,2020,0,7,10,...,0,0,0,,,,Oklahoma,College Stats,/players/W/WillTr21.htm,http://www.sports-reference.com/cfb/players/tr...
4,1,5,KAN,Eric Berry,DB,21,2018,3,5,5,...,0,0,0,377,14,5.5,Tennessee,College Stats,/players/B/BerrEr99.htm,http://www.sports-reference.com/cfb/players/er...


In [12]:
# Create an empty list that will contain all the dataframes
# (one dataframe for each draft)
draft_dfs_list = []

# a list to store any errors that may come up while scraping
errors_list = []

In [14]:
# The url template that we pass in the draft year inro
url_template = "http://www.pro-football-reference.com/years/{year}/draft.htm"

years = range(2010,2016)
# for each year from 1967 to (and including) 2016
for year in years: 
    
    # Use try/except block to catch and inspect any urls that cause an error
    try:
        # get the draft url
        url = url_template.format(year=year)

        # get the html
        html = urlopen(url)

        # create the BeautifulSoup object
        soup = BeautifulSoup(html, "lxml") 

        # get the column headers
        column_headers = [th.getText() for th in 
                          soup.findAll('tr', limit=2)[1].findAll('th')]
        column_headers.extend(["Player_NFL_Link", "Player_NCAA_Link"])

        # select the data from the table using the '#drafts tr' CSS selector
        table_rows = soup.select("#drafts tr")[2:] 

        # extract the player data from the table rows
        player_data = extract_player_data(table_rows)

        # create the dataframe for the current years draft
        year_df = pd.DataFrame(player_data, columns=column_headers)

        # if it is a draft from before 1994 then add a Tkl column at the 
        # 24th position
        if year < 1994:
            year_df.insert(24, "Tkl", "")

        # add the year of the draft to the dataframe
        year_df.insert(0, "Draft_Yr", year)

        # append the current dataframe to the list of dataframes
        draft_dfs_list.append(year_df)
    
    except Exception as e:
        # Store the url and the error it causes in a list
        error =[url, e] 
        # then append it to the list of errors
        errors_list.append(error)

In [15]:
len(errors_list)

0

In [16]:
# store all drafts in one DataFrame
draft_df = pd.concat(draft_dfs_list, ignore_index=True)
draft_df.head()

,Draft_Yr,Rnd,Pick,Tm,Player,Pos,Age,To,AP1,PB,...,Rec,Yds,TD,Solo,Int,Sk,College/Univ,,Player_NFL_Link,Player_NCAA_Link
0,2010,1,1,STL,Sam Bradford,QB,22,2018,0,0,...,1,5,0,,,,Oklahoma,College Stats,/players/B/BradSa00.htm,http://www.sports-reference.com/cfb/players/sa...
1,2010,1,2,DET,Ndamukong Suh,DT,23,2020,3,5,...,0,0,0,368,1,63.5,Nebraska,College Stats,/players/S/SuhxNd99.htm,http://www.sports-reference.com/cfb/players/nd...
2,2010,1,3,TAM,Gerald McCoy,DT,22,2019,1,6,...,0,0,0,235,,59.5,Oklahoma,College Stats,/players/M/McCoGe99.htm,http://www.sports-reference.com/cfb/players/ge...
3,2010,1,4,WAS,Trent Williams,T,22,2020,0,7,...,0,0,0,,,,Oklahoma,College Stats,/players/W/WillTr21.htm,http://www.sports-reference.com/cfb/players/tr...
4,2010,1,5,KAN,Eric Berry,DB,21,2018,3,5,...,0,0,0,377,14,5.5,Tennessee,College Stats,/players/B/BerrEr99.htm,http://www.sports-reference.com/cfb/players/er...


In [18]:
df_qbs = draft_df[draft_df['Pos'] == 'QB']
df_qbs = df_qbs.reset_index(drop=True)
df_qbs

,Draft_Yr,Rnd,Pick,Tm,Player,Pos,Age,To,AP1,PB,...,Rec,Yds,TD,Solo,Int,Sk,College/Univ,,Player_NFL_Link,Player_NCAA_Link
0,2010,1,1,STL,Sam Bradford,QB,22,2018,0,0,...,1,5,0,,,,Oklahoma,College Stats,/players/B/BradSa00.htm,http://www.sports-reference.com/cfb/players/sa...
1,2010,1,25,DEN,Tim Tebow,QB,23,2012,0,0,...,0,0,0,1,,,Florida,College Stats,/players/T/TeboTi00.htm,http://www.sports-reference.com/cfb/players/ti...
2,2010,2,48,CAR,Jimmy Clausen,QB,22,2015,0,0,...,0,0,0,,,,Notre Dame,College Stats,/players/C/ClauJi00.htm,http://www.sports-reference.com/cfb/players/ji...
3,2010,3,85,CLE,Colt McCoy,QB,24,2020,0,0,...,2,8,0,,,,Texas,College Stats,/players/M/McCoCo00.htm,http://www.sports-reference.com/cfb/players/co...
4,2010,4,122,PHI,Mike Kafka,QB,23,2014,0,0,...,0,0,0,,,,Northwestern,College Stats,/players/K/KafkMi00.htm,http://www.sports-reference.com/cfb/players/mi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,2015,3,75,NOR,Garrett Grayson,QB,24,2015,0,0,...,0,0,0,,,,Colorado St.,College Stats,/players/G/GrayGa00.htm,http://www.sports-reference.com/cfb/players/ga...
64,2015,3,89,STL,Sean Mannion,QB,23,2019,0,0,...,0,0,0,,,,Oregon St.,College Stats,/players/M/MannSe00.htm,http://www.sports-reference.com/cfb/players/se...
65,2015,4,103,NYJ,Bryce Petty,QB,24,2017,0,0,...,0,0,0,,,,Baylor,College Stats,/players/P/PettBr01.htm,http://www.sports-reference.com/cfb/players/br...
66,2015,5,147,GNB,Brett Hundley,QB,22,2019,0,0,...,1,10,0,,,,UCLA,College Stats,/players/H/HundBr00.htm,http://www.sports-reference.com/cfb/players/br...


In [24]:
rows = ['Draft_Yr', 'Rnd', 'Pick', 'Tm', 'Player', 'Pos', 'Age', 'To', 'St', 'G', 'College/Univ', 'Player_NFL_Link', 'Player_NCAA_Link']
df_qbs = df_qbs[rows]
df_qbs

,Draft_Yr,Rnd,Pick,Tm,Player,Pos,Age,To,St,G,College/Univ,Player_NFL_Link,Player_NCAA_Link
0,2010,1,1,STL,Sam Bradford,QB,22,2018,5,83,Oklahoma,/players/B/BradSa00.htm,http://www.sports-reference.com/cfb/players/sa...
1,2010,1,25,DEN,Tim Tebow,QB,23,2012,1,35,Florida,/players/T/TeboTi00.htm,http://www.sports-reference.com/cfb/players/ti...
2,2010,2,48,CAR,Jimmy Clausen,QB,22,2015,1,21,Notre Dame,/players/C/ClauJi00.htm,http://www.sports-reference.com/cfb/players/ji...
3,2010,3,85,CLE,Colt McCoy,QB,24,2020,2,43,Texas,/players/M/McCoCo00.htm,http://www.sports-reference.com/cfb/players/co...
4,2010,4,122,PHI,Mike Kafka,QB,23,2014,0,4,Northwestern,/players/K/KafkMi00.htm,http://www.sports-reference.com/cfb/players/mi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,2015,3,75,NOR,Garrett Grayson,QB,24,2015,0,1,Colorado St.,/players/G/GrayGa00.htm,http://www.sports-reference.com/cfb/players/ga...
64,2015,3,89,STL,Sean Mannion,QB,23,2019,0,13,Oregon St.,/players/M/MannSe00.htm,http://www.sports-reference.com/cfb/players/se...
65,2015,4,103,NYJ,Bryce Petty,QB,24,2017,0,10,Baylor,/players/P/PettBr01.htm,http://www.sports-reference.com/cfb/players/br...
66,2015,5,147,GNB,Brett Hundley,QB,22,2019,1,18,UCLA,/players/H/HundBr00.htm,http://www.sports-reference.com/cfb/players/br...


In [25]:
 cleaned = pd.read_csv('data/draft_qbs_2010-2015.csv')
 cleaned.head()

,Unnamed: 0,Draft_Yr,Rnd,Pick,Tm,Player,Pos,Age,To,St,G,College/Univ,Player_NFL_Link,Player_NCAA_Link
0,0,2010,1,1,STL,Sam Bradford,QB,22,2018.0,5,83.0,Oklahoma,/players/B/BradSa00.htm,http://www.sports-reference.com/cfb/players/sa...
1,1,2010,1,25,DEN,Tim Tebow,QB,23,2012.0,1,35.0,Florida,/players/T/TeboTi00.htm,http://www.sports-reference.com/cfb/players/ti...
2,2,2010,2,48,CAR,Jimmy Clausen,QB,22,2015.0,1,21.0,Notre Dame,/players/C/ClauJi00.htm,http://www.sports-reference.com/cfb/players/ji...
3,3,2010,3,85,CLE,Colt McCoy,QB,24,2020.0,2,43.0,Texas,/players/M/McCoCo00.htm,http://www.sports-reference.com/cfb/players/co...
4,4,2010,4,122,PHI,Mike Kafka,QB,23,2014.0,0,4.0,Northwestern,/players/K/KafkMi00.htm,http://www.sports-reference.com/cfb/players/mi...


In [26]:
cleaned.shape

(68, 14)